# 01 — FVM Sod Shock Tube

Comparação entre a solução numérica via volumes finitos (Rusanov) e a solução analítica do problema de Sod.

In [ ]:
import sys
from pathlib import Path

PROJECT_ROOT = Path('..').resolve()
SRC_PATH = PROJECT_ROOT / 'src'
if str(SRC_PATH) not in sys.path:
    sys.path.insert(0, str(SRC_PATH))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from riemann_ml.core.euler1d import StatePrim
from riemann_ml.exact.sod_exact import sod_exact_profile
from riemann_ml.fvm.solver import simulate

GAMMA = 1.4
NUM_CELLS = 200
FINAL_TIME = 0.2
CFL = 0.5
INTERFACE = 0.5

left_state = StatePrim(density=1.0, velocity=0.0, pressure=1.0)
right_state = StatePrim(density=0.125, velocity=0.0, pressure=0.1)

times, x, q, _ = simulate(
    num_cells=NUM_CELLS,
    final_time=FINAL_TIME,
    cfl=CFL,
    left_state=left_state,
    right_state=right_state,
    gamma=GAMMA,
    interface_position=INTERFACE,
    store_history=False,
)

rho = q[:, 0]
momentum = q[:, 1]
energy = q[:, 2]
velocity = momentum / rho
pressure = (GAMMA - 1.0) * np.maximum(energy - 0.5 * momentum**2 / np.clip(rho, 1e-12, None), 1e-12)

rho_exact, u_exact, p_exact = sod_exact_profile(
    x - INTERFACE,
    FINAL_TIME,
    left_state=left_state,
    right_state=right_state,
    gamma=GAMMA,
)

fig, axes = plt.subplots(3, 1, figsize=(10, 9), sharex=True)
axes[0].plot(x, rho, label='FVM')
axes[0].plot(x, rho_exact, '--', label='Exact')
axes[0].set_ylabel('Density')
axes[0].grid(True)
axes[0].legend()

axes[1].plot(x, velocity, label='FVM')
axes[1].plot(x, u_exact, '--', label='Exact')
axes[1].set_ylabel('Velocity')
axes[1].grid(True)

axes[2].plot(x, pressure, label='FVM')
axes[2].plot(x, p_exact, '--', label='Exact')
axes[2].set_ylabel('Pressure')
axes[2].set_xlabel('x')
axes[2].grid(True)

fig.suptitle(f'Sod shock tube: FVM vs exact (t = {FINAL_TIME:.3f})')
fig.tight_layout()
plt.show()

## Artefatos gerados

A célula abaixo exibe a figura produzida pelos scripts de sanidade ou avaliação (quando disponível).


In [ ]:
from pathlib import Path
from IPython.display import Image, display

figure_path = Path('data/artifacts/eval/sod/sod_comparison.png')
if figure_path.exists():
    display(Image(filename=str(figure_path)))
else:
    print('Figura não encontrada:', figure_path)


In [ ]:
import json
from pathlib import Path

metrics_path = Path('data/artifacts/eval/sod/sod_metrics.json')
if metrics_path.exists():
    metrics = json.loads(metrics_path.read_text(encoding='utf-8'))
    fvm_metrics = metrics.get('FVM', {})
    print('Métricas FVM vs. exata:')
    for key, value in fvm_metrics.items():
        print(f'  {key}: {value:.6f}')
else:
    print('Arquivo de métricas não encontrado:', metrics_path)


> **Comentário:** observe as diferenças de densidade, velocidade e pressão entre o método numérico e a solução exata; os valores numéricos ajudam a documentar o desvio relativo.
